Content filtering

Collabrative filtering

Bara tutorial: https://www.geeksforgeeks.org/build-a-recommendation-engine-with-collaborative-filtering/

Prenseter i notebook eller i dash app. Hur man prensntera en lista på topp 5 av dom man blir recomenderad. Om vi använder den simpla vägen med bara ratings. 

Använder vi tagsen från användaren. Då får vi jobba mer med texten där man får bygga ett dokumnet för varje användare och hitta likheter mellan tagar mellan olika filmer. 

Recomandering skulle också komma från att kunna förutse vad användaren skulle kunna betygsätta en film. Så göra en förutsägelse och recomandera den som toppar förutsägelsen. 



In [3]:
import pandas as pd

data_games  = pd.read_csv("../data/steam-200k.csv", names= ["UserId", "Games", "Purchase", "Value", "Unused"])
data_games

,UserId,Games,Purchase,Value,Unused
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0
...,...,...,...,...,...
199995,128470551,Titan Souls,play,1.5,0
199996,128470551,Grand Theft Auto Vice City,purchase,1.0,0
199997,128470551,Grand Theft Auto Vice City,play,1.5,0
199998,128470551,RUSH,purchase,1.0,0


Hur går vi väga förat att bygga en feature mängde här. En sak vi skulle kunna göra är att sammla alla tid som en unik spelar har för varje spel. Så får man ett medel. Efter det kan man också hitta medlet över alla spelar för spelet och hitta ett medel. Sedan titta på outliers mot medlen. Skulle kunna var ett sätt att hitta spelar som man skulle kunna recomandera spel mot. 

Collabratvie filtering. Handlar om vad andra användare tyckt. Man försöker hitta likeheter och mönster i det. 

In [4]:
data_consumer = pd.read_csv("../data/user_personalized_features.csv", index_col=0)

data_consumer.head()

,User_ID,Age,Gender,Location,Income,Interests,Last_Login_Days_Ago,Purchase_Frequency,Average_Order_Value,Total_Spending,Product_Category_Preference,Time_Spent_on_Site_Minutes,Pages_Viewed,Newsletter_Subscription
0,#1,56,Male,Suburban,38037,Sports,5,7,18,2546,Books,584,38,True
1,#2,46,Female,Rural,103986,Technology,15,7,118,320,Electronics,432,40,False
2,#3,32,Female,Suburban,101942,Sports,28,1,146,3766,Apparel,306,1,True
3,#4,60,Female,Suburban,71612,Fashion,18,3,163,4377,Apparel,527,29,False
4,#5,25,Male,Suburban,49725,Travel,2,5,141,4502,Health & Beauty,53,10,True


In [5]:
data_moive = pd.read_csv("../data/ml-latest-small/movies.csv")

data_moive

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [6]:
data_moive["genres"]

0       Adventure|Animation|Children|Comedy|Fantasy
1                        Adventure|Children|Fantasy
2                                    Comedy|Romance
3                              Comedy|Drama|Romance
4                                            Comedy
                           ...                     
9737                Action|Animation|Comedy|Fantasy
9738                       Animation|Comedy|Fantasy
9739                                          Drama
9740                               Action|Animation
9741                                         Comedy
Name: genres, Length: 9742, dtype: object

In [7]:
data_moive_rating = pd.read_csv("../data/ml-latest-small/ratings.csv")



"""
Vissa användare kanske lägger viktade betyg. Det finns massa psykologiska användar mönster.
Men blir mindre viktet när datan blir större. Fasst man kan inte vet innan man testa. Enklast är att använda
ratings och gener för att hitta recomendationer. 
"""

'\nVissa användare kanske lägger viktade betyg. Det finns massa psykologiska användar mönster.\nMen blir mindre viktet när datan blir större. Fasst man kan inte vet innan man testa. Enklast är att använda\nratings och gener för att hitta recomendationer. \n'

In [8]:
data_movie_tages = pd.read_csv("../data/ml-latest-small/tags.csv")

data_movie_tages[data_movie_tages["userId"] == 2]

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
5,2,89774,Tom Hardy,1445715205
6,2,106782,drugs,1445715054
7,2,106782,Leonardo DiCaprio,1445715051
8,2,106782,Martin Scorsese,1445715056


In [9]:
data_books = pd.read_csv("../data/Books.csv")
data_books_ratings = pd.read_csv("../data/Ratings.csv")
data_books_users = pd.read_csv("../data/Users.csv")

C:\Users\User\AppData\Local\Temp\ipykernel_16028\595871367.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data_books = pd.read_csv("../data/Books.csv")


In [10]:
data_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [11]:
data_books_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [12]:
data_books_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [13]:
data_books[data_books["Book-Title"].duplicated()].sort_values("Book-Title")

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
156341,1565920317,!%@ (A Nutshell handbook),Donnalyn Frey,1993,O'Reilly,http://images.amazon.com/images/P/1565920317.0...,http://images.amazon.com/images/P/1565920317.0...,http://images.amazon.com/images/P/1565920317.0...
158204,0792277295,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,2001,National Geographic,http://images.amazon.com/images/P/0792277295.0...,http://images.amazon.com/images/P/0792277295.0...,http://images.amazon.com/images/P/0792277295.0...
22024,067103975X,'Salem's Lot,Stephen King,2000,Pocket,http://images.amazon.com/images/P/067103975X.0...,http://images.amazon.com/images/P/067103975X.0...,http://images.amazon.com/images/P/067103975X.0...
94187,0385007515,'Salem's Lot,Stephen King,1993,Doubleday Books,http://images.amazon.com/images/P/0385007515.0...,http://images.amazon.com/images/P/0385007515.0...,http://images.amazon.com/images/P/0385007515.0...
249713,006440143X,...And Now Miguel,Joseph Krumgold,1984,HarperTrophy,http://images.amazon.com/images/P/006440143X.0...,http://images.amazon.com/images/P/006440143X.0...,http://images.amazon.com/images/P/006440143X.0...
...,...,...,...,...,...,...,...,...
267601,0887306314,"\Yes\"" or \""No\"": The Guide to Better Decisions""",Spencer Johnson,1993,HarperBusiness,http://images.amazon.com/images/P/0887306314.0...,http://images.amazon.com/images/P/0887306314.0...,http://images.amazon.com/images/P/0887306314.0...
109636,0312113536,"\You're Okay, It's Just a Bruise\"": A Doctor's...",Rob Huizenga,1994,St Martins Pr,http://images.amazon.com/images/P/0312113536.0...,http://images.amazon.com/images/P/0312113536.0...,http://images.amazon.com/images/P/0312113536.0...
193436,0316507490,dot.bomb: My Days and Nights at an Internet Go...,J. David Kuo,2001,"Little, Brown",http://images.amazon.com/images/P/0316507490.0...,http://images.amazon.com/images/P/0316507490.0...,http://images.amazon.com/images/P/0316507490.0...
193457,0316089702,dot.bomb: My Days and Nights at an Internet Go...,J. David Kuo,2002,Back Bay Books,http://images.amazon.com/images/P/0316089702.0...,http://images.amazon.com/images/P/0316089702.0...,http://images.amazon.com/images/P/0316089702.0...


In [14]:
new_data_book = data_books.drop_duplicates(["Book-Title"])

In [15]:
rating_with_name = data_books_ratings.merge(new_data_book, on="ISBN")
rating_with_name.drop(["ISBN", "Image-URL-S", "Image-URL-M", "Image-URL-L", "Publisher"], axis=1, inplace=True)

In [16]:
rating_with_name.dropna(inplace=True)

In [17]:
rating_with_name.info()

<class 'pandas.core.frame.DataFrame'>
Index: 883077 entries, 0 to 883078
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   User-ID              883077 non-null  int64 
 1   Book-Rating          883077 non-null  int64 
 2   Book-Title           883077 non-null  object
 3   Book-Author          883077 non-null  object
 4   Year-Of-Publication  883077 non-null  object
dtypes: int64(2), object(3)
memory usage: 40.4+ MB


In [25]:
x = rating_with_name.groupby("User-ID").count()["Book-Rating"] > 100
expert_users = x[x].index

filtered_user_ratings = rating_with_name[rating_with_name["User-ID"].isin(expert_users)]

y = filtered_user_ratings.groupby("Book-Title").count()["Book-Rating"] >= 50
famouse_books = y[y].index
user_ratings = filtered_user_ratings[filtered_user_ratings["Book-Title"].isin(famouse_books)]


In [27]:
design_matrix = user_ratings.pivot_table(index="Book-Title", columns= "User-ID", values="Book-Rating")
design_matrix.fillna(0, inplace=True)

design_matrix.head()

User-ID,254,507,882,1424,1435,1733,1903,2033,2110,2276,...,274808,275020,275970,276680,277427,277478,277639,278137,278188,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010: Odyssey Two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

scaler = StandardScaler(with_mean=True, with_std=True)

design_matrix_centered = scaler.fit_transform(design_matrix)

In [32]:
similarity_score = cosine_similarity(design_matrix_centered)

similarity_score

array([[ 1.        , -0.01859849,  0.08733311, ...,  0.01350255,
        -0.02201299, -0.02261981],
       [-0.01859849,  1.        , -0.0294832 , ..., -0.02238366,
        -0.03070822,  0.10549892],
       [ 0.08733311, -0.0294832 ,  1.        , ...,  0.00231518,
         0.00643198, -0.01337545],
       ...,
       [ 0.01350255, -0.02238366,  0.00231518, ...,  1.        ,
        -0.00500822, -0.01371862],
       [-0.02201299, -0.03070822,  0.00643198, ..., -0.00500822,
         1.        ,  0.00914464],
       [-0.02261981,  0.10549892, -0.01337545, ..., -0.01371862,
         0.00914464,  1.        ]], shape=(620, 620))

In [38]:
import numpy as np

def recommend(book_name):
    index = np.where(design_matrix.index==book_name)[0][0]
    similar_books = sorted(list(enumerate(similarity_score[index])), key=lambda x: x[1], reverse=True)[1:6]
    data = []

    for index, similarity in similar_books:
        item = []
        temp_df =new_data_book[new_data_book["Book-Title"] == design_matrix.index[index]]
        item.extend(temp_df["Book-Title"].values)
        item.extend(temp_df["Book-Author"].values)
        data.append(item)

    return data

In [40]:
recommend("Animal Farm")

[['1984', 'George Orwell'],
 ['The Two Dead Girls (Green Mile Series)', 'Stephen King'],
 ['From a Buick 8 : A Novel', 'Stephen King'],
 ['A Walk in the Woods: Rediscovering America on the Appalachian Trail (Official Guides to the Appalachian Trail)',
  'Bill Bryson'],
 ['Tis : A Memoir', 'Frank McCourt']]